In [40]:
import sys, os, distutils.core
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)
print(torch.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Feb__8_05:53:42_Coordinated_Universal_Time_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0
torch:  2.1 ; cuda:  cu118
detectron2: 0.6
2.1.0.dev20230317+cu118


In [41]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger

setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
# from google.colab.patches import cv2_imshow
import cv2

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [42]:
# im = cv2.imread("../data/example/000000439715.jpg")
# im = cv2.imread("../data/example/tim_paddle_135.jpg")
# im = cv2.imread("../data/example/fendy_tim_inholland_780.jpg")
# im = cv2.imread("../data/example/prevgroup_vid9_220.jpg")
im = cv2.imread("../data/example/Stilleben-Heem-Jan-Davidsz-de-oil-painting-1.jpg")
# cv2.imshow('', im)
# cv2.waitKey(0)

In [43]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

[03/17 18:41:17 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


In [44]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

tensor([75, 45, 49, 47, 50, 47, 75, 55, 49, 47, 49, 48, 47, 49, 50, 47, 13, 49,
        60, 45, 60, 49, 47], device='cuda:0')
Boxes(tensor([[  99.3056,  959.7385,  603.4999, 1233.0151],
        [ 660.8810,  447.1964,  975.2817,  583.7662],
        [ 158.7107,  807.8477,  324.9533,  932.1846],
        [ 672.6257,  445.2398,  961.2783,  558.9960],
        [ 630.7598,  181.6343,  786.7418,  328.6764],
        [1233.3925,  476.4240, 1331.0933,  581.0875],
        [ 118.4252, 1004.2790,  264.9449, 1229.1565],
        [ 693.2082,  610.8507, 1124.2042,  837.5369],
        [ 304.0574,  776.5294,  436.9491,  897.9233],
        [ 296.8392,  776.2922,  436.9153,  901.7314],
        [1235.4055,  477.8568, 1338.1301,  583.2129],
        [ 694.8641,  609.2538, 1123.8347,  838.4774],
        [ 675.6906,  459.9525,  804.4937,  559.1673],
        [1320.1996,  496.5484, 1400.5161,  579.4964],
        [ 702.6865,  183.1704,  782.0750,  315.1305],
        [1124.9364,  406.0874, 1209.0831,  472.7047],
    

In [45]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imshow('', out.get_image()[:, :, ::-1])
cv2.waitKey(0)

-1

In [46]:
%load_ext tensorboard
%tensorboard --logdir runs

Launching TensorBoard...